In [8]:
import numpy as np
import scipy as sp
import pandas as pd
import datetime

from compute import *

In [9]:
#constants
ProjectName = "2912 East 18th street 5plex"

# assumptions
NumberUnits=5
price     = 1250000
rent      = 1110+1350+1300+1465+1675
Carry     = 0
ltv       = 65
loanRatePer = 3.5
expenseRate = 36
# no adus assumed yet, just revamp
# Proforma
Description  = "Raise rents minimal renovation"
rentPF       = 2100*1+1600*4
expenseRatePF= 32.5
capPF        = 4.5
repairCost   = 5000*5
computeProjectSimple(ProjectName, NumberUnits, price, rent,expenseRate, Carry,ltv, loanRatePer, rentPF,expenseRatePF,capPF,repairCost,
                    description=Description)

# Proforma
Description  = "Add two adus"
rentPF       = 2100*1+1600*6
expenseRatePF= 32.5
capPF        = 4.5
repairCost   = 10000*5+300000
computeProjectSimple(ProjectName, NumberUnits, price, rent,expenseRate, Carry,ltv, loanRatePer, rentPF,expenseRatePF,capPF,repairCost,
                    description=Description,
                    show=2)


# Proforma
Description  = "21 units project R(19)+C(2)"
rentPF       = 19*1800+2*1600*1.5
expenseRatePF= 32.5
capPF        = 4.5
repairCost   = 150*850*19+150*1600*2 + 500000
computeProjectSimple(ProjectName, NumberUnits, price, rent,expenseRate, Carry,ltv, loanRatePer, rentPF,expenseRatePF,capPF,repairCost,
                    description=Description,
                    show=2)




        Project   : 2912 East 18th street 5plex
        Purchase  : $1,250,000
        rent roll : $82,800
        NOI       : $52,992  
        Expen Rate: 36%
        Cap Rate  : 4.24%        
        DebService:
           amount - $812,500
           rate   - 3.5%
           payment- 43,782$
        DSCR      : 1.21%  
        
    
        Descript  : Raise rents minimal renovation
        Basis     : $1,275,000
        Cap Rate  : 5.40%        
        rent roll : $102,000
        NOI       : $68,850
        Expen Rate: 32.5%    
        Price     : $1,530,000 @ cap(4.50%)
        
    
        Descript  : Add two adus
        Basis     : $1,600,000
        Cap Rate  : 5.92%        
        rent roll : $140,400
        NOI       : $94,770
        Expen Rate: 32.5%    
        Price     : $2,106,000 @ cap(4.50%)
        
    
        Descript  : 21 units project R(19)+C(2)
        Basis     : $4,652,500
        Cap Rate  : 6.79%        
        rent roll : $468,000
        NOI   

In [3]:
dfAct = pd.DataFrame(columns=["Value","Monthly","Annual","%"])

dfAct.loc["Purchase Price","Value"] = price
dfAct.loc["closing cost","%"]       = 1
dfAct.loc["Repair","Value"]         = 5000
dfAct.loc["Reserve","Value"]        = 6000*5
dfAct.loc["Rent Roll","Monthly"] = rent
dfAct.loc["Loan Rate","%"]  = 3.5
dfAct.loc["LTV","%"]        = 75
dfAct.loc["Number Of Units","Value"]=5
simpleExpense = 1
dfAct.loc["Expenses","%"]   =30
dfAct.loc["Util Per Unit","Value"]  =0
dfAct.loc["Insurance","%"]  =0.2
dfAct.loc["Taxes","%"]      = 1.25
dfAct.loc["Vacancy","%"]    = 4
dfAct.loc["Management","%"] = 5
dfAct.loc["Maintenance","%"]= 0.3

In [4]:
#derived from tower

dfact = setMonthlyAndAnnual(dfAct)
if simpleExpense ==1:
    noi=computeNoi(rent=dfAct.loc["Rent Roll","Monthly"],expenseRate=dfAct.loc["Expenses","%"])
    dfAct.loc["NOI","Annual"] = noi
    dfAct.loc["Expenses","Monthly"]=dfAct.loc["Expenses","%"]/100. * dfAct.loc["Rent Roll","Monthly"]
else:
    dfAct.loc["Util","Monthly"] = dfAct.loc["Number Of Units","Value"] * dfAct.loc["Util Per Unit","Value"]
    for c in ["Vacancy","Management"]:
        dfAct.loc[c,"Monthly"] = dfAct.loc[c,"%"]/100.*dfAct.loc["Rent Roll","Monthly"]
        dfAct.loc[c,"Monthly"] = dfAct.loc[c,"%"]/100.*dfAct.loc["Rent Roll","Monthly"]
    for c in ["Maintenance","Taxes","Insurance"]:
        dfAct.loc[c,"Monthly"]= dfAct.loc[c,"%"]/100.*dfAct.loc["Purchase Price","Value"]/12.
    dfAct.loc["Expenses","Monthly"] = sum(dfAct.loc[["Vacancy","Management","Maintenance","Taxes","Insurance","Util"],"Monthly"])
    dfAct.loc["NOI","Monthly"]      = dfAct.loc["Rent Roll","Monthly"] - dfAct.loc["Expenses","Monthly"]
    dfAct.loc["Expense","%"] = dfAct.loc["Expenses","Monthly"]/ dfAct.loc["Rent Roll","Monthly"]
    
dfAct.loc["closing cost","Value"]=dfAct.loc["Purchase Price","Value"] * dfAct.loc["closing cost","%"]/100.
dfAct.loc["Cap","%"] = computeCap(
    rent    =dfAct.loc["Rent Roll","Annual"],
    price   =dfAct.loc["Repair","Value"]+dfAct.loc["Purchase Price","Value"] ,
    expenses=dfAct.loc["Expenses","Annual"])    

# loan payment
pmt =  -1*np.pmt(
    dfAct.loc["Loan Rate","%"]/100/12.,
    360,dfAct.loc["Purchase Price","Value"] *dfAct.loc["LTV","%"]/100)
dfAct.loc["Debt Service","Monthly"] = pmt
dfAct.loc["DSCR","%"] = dfAct.loc["NOI","Monthly"] / dfAct.loc["Debt Service","Monthly"] 
dfact = setMonthlyAndAnnual(dfAct)

dfAct.loc["Cash Flow","Annual"] = (dfAct.loc["NOI","Annual"] - dfAct.loc["Debt Service","Annual"]) 

dfAct.loc["Out Of Pocket","Value"]=\
    dfAct.loc["closing cost","Value"]+\
    dfAct.loc["Repair","Value"]+\
    dfAct.loc["Reserve","Value"]+\
    dfAct.loc["Purchase Price","Value"]*(100-dfAct.loc["LTV","%"])/100.
dfAct.loc["Cash Flow","%"] = dfAct.loc["Cash Flow","Annual"] / dfAct.loc["Out Of Pocket","Value"] * 100
dfact = setMonthlyAndAnnual(dfAct)
dfAct

,Value,Monthly,Annual,%
Purchase Price,1250000,NaN,NaN,NaN
closing cost,12500,NaN,NaN,1
Repair,5000,NaN,NaN,NaN
Reserve,30000,NaN,NaN,NaN
Rent Roll,NaN,6900,82800,NaN
Loan Rate,NaN,NaN,NaN,3.5
LTV,NaN,NaN,NaN,75
Number Of Units,5,NaN,NaN,NaN
Expenses,NaN,2070,24840,30
Util Per Unit,0,NaN,NaN,NaN


In [5]:
computeCap(
    rent    =dfAct.loc["Rent Roll","Annual"],
    price   =1.3e6,
    expenses=dfAct.loc["Expenses","Annual"])    


4.458461538461538

In [6]:
simpleDepreciation=1
rentIncrease=5        # rent increase
maintenanceIncrease=3 # for all OI
years =datetime.datetime.today().year + np.arange(0,10)
dfPF = pd.DataFrame(columns=years)

updates = {'year':1,"expenseRate":30,"repairCost":450000, "rent":2200*2+1500*2+2000*2}

for year in years:
    if year == years[0]:
        dfPF.loc["Rent Roll",year]    = dfAct.loc["Rent Roll","Annual"]
        dfPF.loc["Expenses", year]    = -1*dfAct.loc["Expenses","Annual"]
    elif year == years[0] +updates["year"]:
        dfPF.loc["Rent Roll",year]    = updates["rent"]*12
        dfPF.loc["Expenses",year]     = -1*dfPF.loc["Rent Roll",year] * updates["expenseRate"]/100.
    else:
        dfPF.loc["Rent Roll",year]    = dfPF.loc["Rent Roll",year-1] *(100+rentIncrease)/100.
        dfPF.loc["Expenses",year]     = dfPF.loc["Expenses",year-1]  *(100+maintenanceIncrease)/100.
        
    dfPF.loc["NOI",year]          = dfPF.loc["Rent Roll",year]+dfPF.loc["Expenses",year]
    dfPF.loc["Debt Service",year] = computePmt(dfAct,year=1)*12
    if simpleDepreciation == 1:
        dfPF.loc["Depreciation",year] = (dfAct.loc["Repair","Value"]+dfAct.loc["Purchase Price","Value"])* -2./3./27.5    
    # Cash Flow NOI minus debt service
    dfPF.loc["Cash Flow",year]    = dfPF.loc["NOI",year] + dfPF.loc["Debt Service",year]
    # Profit=NOI minus interest payment
    dfPF.loc["Profit",year]       = dfPF.loc["NOI",year] + computePmt(dfAct,year=1,ptype=LoanPaymentType.Interest)*12    
    # Pre Tax Profit = profit - minus depreciation
    dfPF.loc["Pre-Tax Profit",year]   = dfPF.loc["Profit",year] + dfPF.loc["Depreciation",year]
    dfPF.loc["Cap Rate",year]         = dfPF.loc["NOI",year]      / dfAct.loc["Purchase Price","Value"]*100
    # cash on cash = profit / out of pocket acquisition
    dfPF.loc["Cash on Cash",year]     = dfPF.loc["Profit",year]/ dfAct.loc["Out Of Pocket","Value"] * 100
    #dfPF.loc["Cash Flow","%"] = dfPF.loc["Cash Flow","Annual"] / dfAct.loc["Out Of Pocket","Value"] * 100
dfPF

,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029
Rent Roll,82800,136800,143640,150822,158363,166281,174595,183325,192491,202116
Expenses,-24840,-41040,-42271.2,-43539.3,-44845.5,-46190.9,-47576.6,-49003.9,-50474,-51988.2
NOI,57960,95760,101369,107283,113518,120090,127019,134321,142017,150128
Debt Service,-50517.5,-50517.5,-50517.5,-50517.5,-50517.5,-50517.5,-50517.5,-50517.5,-50517.5,-50517.5
Depreciation,-30424.2,-30424.2,-30424.2,-30424.2,-30424.2,-30424.2,-30424.2,-30424.2,-30424.2,-30424.2
Cash Flow,7442.47,45242.5,50851.3,56765.1,63000.1,69572.8,76501.2,83803.7,91499.8,99610.1
Profit,25147.5,62947.5,68556.3,74470.2,80705.1,87277.9,94206.2,101509,109205,117315
Pre-Tax Profit,-5276.74,32523.3,38132.1,44045.9,50280.8,56853.6,63782,71084.4,78780.6,86890.9
Cap Rate,4.6368,7.6608,8.1095,8.58261,9.08141,9.60723,10.1615,10.7457,11.3614,12.0102
Cash on Cash,6.98542,17.4854,19.0434,20.6862,22.4181,24.2439,26.1684,28.1969,30.3347,32.5875
